### 1. 什么是 RNN（循环神经网络）？

RNN（Recurrent Neural Network）是一类用于建模**序列数据**的神经网络，广泛应用于自然语言处理、时间序列分析、语音识别等任务。

与前馈神经网络（如 MLP、CNN）不同，RNN 具有“记忆性”：
> **RNN 能够使用先前输入的信息来影响当前的输出**，本质上具备“时间维度”的建模能力。

---

#### 🔁 为什么叫“循环”？
RNN 会将每一个时间步的输出作为“隐藏状态”传递给下一个时间步：

$$
h_t = f(W_{xh}x_t + W_{hh}h_{t-1} + b)
$$

其中：
- $x_t$：当前时间步的输入
- $h_t$：当前时间步的隐藏状态（等价于输出）
- $h_{t-1}$：上一时间步的隐藏状态
- $W_{xh}, W_{hh}$：输入和隐藏状态的权重矩阵
- $f$：非线性函数（如 tanh 或 ReLU）

你可以理解为每一时刻的神经网络“共享参数”并连接在时间轴上。

---

#### ⏳ 序列建模的输入输出模式

| 模式类型         | 应用示例                     |
|------------------|------------------------------|
| 多对一（Many to One） | 文本情感分类（输入是句子，输出是标签） |
| 多对多（Many to Many） | 机器翻译、语音识别               |

---

#### 🧠 举个例子：情感分类任务
输入序列：`[i, love, this, movie]` → 输出：`positive`

RNN 会依次读取每个单词，更新隐藏状态：
```text
h_1 = f(x_1)
↓
h_2 = f(x_2, h_1)
↓
h_3 = f(x_3, h_2)
↓
h_4 = f(x_4, h_3) → 输出分类结果
```

---

#### ✅ 总结一句话：
> RNN 通过时间步之间的隐藏状态传递，实现了对序列上下文的建模，是处理“有顺序”数据的关键结构。

### 2. RNN 的前向传播机制与状态计算

循环神经网络的核心特性是**具有记忆机制的状态传播**，即隐藏状态 $\mathbf{h}_t$ 能够携带前一时间步的信息 $\mathbf{h}_{t-1}$，实现序列建模。

---

#### 🔁 单层 RNN 的基本公式（D2L 标准记法）
给定输入序列 $\mathbf{X} = (\mathbf{x}_1, \dots, \mathbf{x}_T)$，隐藏状态为 $\mathbf{h}_t$，输出为 $\mathbf{o}_t$，则：

- 隐藏状态更新：
$$
\mathbf{h}_t = \tanh(\mathbf{W}_{xh}\mathbf{x}_t + \mathbf{W}_{hh}\mathbf{h}_{t-1} + \mathbf{b}_h)
$$

- 输出计算：
$$
\mathbf{o}_t = \mathbf{W}_{hq}\mathbf{h}_t + \mathbf{b}_q
$$

其中：
- $\mathbf{W}_{xh}$：输入到隐藏状态的权重矩阵（形状为 $H \times I$）
- $\mathbf{W}_{hh}$：隐藏状态到隐藏状态的权重（$H \times H$）
- $\mathbf{W}_{hq}$：隐藏状态到输出的权重（$O \times H$）
- $\mathbf{b}_h, \mathbf{b}_q$：偏置项
- $H$ 表示隐藏单元维度，$I$ 输入维度，$O$ 输出维度

---

#### ⏳ 状态传播展开（结合上传图）
时间序列被展开为多个时间步的神经网络副本，彼此之间通过 $\mathbf{h}_{t-1} \to \mathbf{h}_t$ 的状态传递连接，所有时间步共享 $\mathbf{W}_{xh}, \mathbf{W}_{hh}, \mathbf{W}_{hq}$。

展开图体现了循环依赖关系：
```
x₁ → h₁ → o₁
     ↓
x₂ → h₂ → o₂
     ↓
x₃ → h₃ → o₃
```
每一层都在使用相同的参数进行递归更新。

---

#### 📐 推导视角：向量化前向传播
- 假设输入序列为一个 batch：$\mathbf{X} \in \mathbb{R}^{T \times I}$，隐藏初始状态 $\mathbf{h}_0 = \mathbf{0}$
- 则可以用循环代码形式推导：
```python
h = h0
for t in range(T):
    h = tanh(X[t] @ W_xh.T + h @ W_hh.T + b_h)
    o = h @ W_hq.T + b_q
```

---

#### 📌 激活函数的选择
- 通常使用 `tanh` 或 `ReLU`
- `tanh` 在 $[-1, 1]$ 范围内压缩值，有助于捕捉平稳动态，但易出现梯度消失（后续模块讲解）

---

#### 🧠 总结一句话：
> RNN 的前向传播通过隐藏状态在时间轴上的递归更新实现序列建模，是捕捉时序依赖关系的基础结构，数学上由带参数共享的非线性递推公式定义。


### 3. RNN 的梯度爆炸与梯度消失问题

RNN 能够捕捉序列中的时间依赖关系，但它也存在一个著名的训练困难：

> **长序列训练时，梯度可能在反向传播过程中不断变大（爆炸）或缩小（消失）**，从而导致模型不收敛或无法学习长期依赖。

---

### 📉 为什么会出现这些问题？

我们回顾隐藏状态更新公式：
$$
\mathbf{h}_t = \tanh(\mathbf{W}_{hh}\mathbf{h}_{t-1} + \mathbf{W}_{xh}\mathbf{x}_t + \mathbf{b}_h)
$$

而在反向传播时，损失函数对早期时间步隐藏状态的梯度依赖于隐藏状态之间的链式求导：
$$
\frac{\partial \mathcal{L}}{\partial \mathbf{h}_{t-k}} = \frac{\partial \mathcal{L}}{\partial \mathbf{h}_t} \cdot \prod_{i=1}^{k} \frac{\partial \mathbf{h}_{t-i+1}}{\partial \mathbf{h}_{t-i}}
$$

也就是隐藏状态之间的导数连乘。

---

### 🔍 梯度消失（Vanishing Gradient）
当每一项导数范数 $\left\|\frac{\partial \mathbf{h}_t}{\partial \mathbf{h}_{t-1}}\right\| < 1$：
- 连乘后指数级衰减，导致梯度趋近 0
- 模型无法更新早期参数 → 忘记远程信息

### 🔥 梯度爆炸（Exploding Gradient）
当导数范数 $> 1$ 且乘积不断放大：
- 梯度在传播中指数增长
- 会导致模型不稳定、Loss 震荡甚至发散

---

### ⚙️ 可视化理解（梯度连乘）
若 $\lambda$ 为每步导数的特征值大小，传播 $k$ 步后梯度为：
$$
\lambda^k
$$
- 若 $\lambda < 1$ → 梯度消失
- 若 $\lambda > 1$ → 梯度爆炸

---

### 🛠 解决方案
| 方法             | 说明 |
|------------------|------|
| 梯度裁剪（Clipping） | 限制最大梯度范数，防止爆炸（`torch.nn.utils.clip_grad_norm_`）|
| 使用 LSTM/GRU     | 特殊结构设计，缓解梯度问题（详见下节）|
| 正则化 / Dropout  | 避免过拟合和不稳定 |
| 层归一化（LayerNorm） | 稳定前向与反向信号 |

---

### 🛠 解决方案
梯度裁剪（Clipping）  限制最大梯度范数，防止爆炸，可通过如下方式实现：
```python
from torch.nn.utils import clip_grad_norm_
clip_grad_norm_(model.parameters(), max_norm=1.0)
```
此外，D2L 书中解释了梯度裁剪的背景：由于梯度爆炸会导致参数更新过大（甚至为 NaN），我们可以在每次参数更新前，**计算所有参数梯度的范数**，并在其超过阈值时按比例缩放，使其保持在一个可控范围内，从而稳定训练过程。具体建议实现如下：

【代码仅为参考】
```python
def grad_clipping(net, theta):
    params = [p for p in net.parameters() if p.requires_grad]
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm
```

| 方法             | 说明 |
|------------------|------|
| 使用 LSTM/GRU     | 特殊结构设计，缓解梯度问题|
| 正则化 / Dropout  | 避免过拟合和不稳定 |
| 层归一化（LayerNorm） | 稳定前向与反向信号 |


### 🧠 总结：
> 梯度爆炸与消失是基础 RNN 难以建模长距离依赖的根本原因。理解它们的来源，是掌握 LSTM、GRU 等更强序列模型的基础。


### 4. 使用 PyTorch 实现基础 RNN（nn.RNN）前向传播

在本模块中，我们将使用 PyTorch 的 `nn.RNN` 模块快速搭建一个基础的循环神经网络，并观察其前向传播的执行过程。

---

### 📦 PyTorch 中的 nn.RNN 模块结构
`nn.RNN(input_size, hidden_size, num_layers)` 是一个内置的标准 RNN 模块：

- `input_size`：每个时间步输入的维度
- `hidden_size`：隐藏状态维度 $\mathbf{h}_t$
- `num_layers`：RNN 堆叠层数（默认 1）

该模块的前向函数：
```python
output, h_n = rnn(x, h_0)
```
- `x`：输入序列，形状为 (seq_len, batch_size, input_size)
- `h_0`：初始隐藏状态，形状为 (num_layers, batch_size, hidden_size)
- `output`：所有时间步的输出 $\mathbf{o}_t$
- `h_n`：最后一个时间步的隐藏状态 $\mathbf{h}_T$

---


In [ ]:
### 🧪 示例代码：构造一个 RNN 前向传播
import torch
from torch import nn

# 输入序列形状: (时间步数, 批量大小, 特征维度)
x = torch.randn(5, 3, 10)  # 5 个时间步，batch=3，输入特征维度=10

# 定义 RNN
rnn = nn.RNN(input_size=10, hidden_size=20, num_layers=1)

# 初始隐藏状态 h0: (层数, batch, 隐藏维度)
h0 = torch.zeros(1, 3, 20)

# 前向传播
output, hn = rnn(x, h0)

print(output.shape)  # (5, 3, 20)，所有时间步的输出
print(hn.shape)      # (1, 3, 20)，最终隐藏状态

### 5. 手动实现 RNN 的前向传播机制（仿照 D2L）

为了更深入理解 RNN 的本质，我们可以仿照 D2L 教程手动实现前向传播过程。这样做有助于你掌握隐藏状态如何从前一状态与当前输入计算得到。

RNN 单步更新公式如下：
$$
\mathbf{h}_t = \tanh(\mathbf{x}_t \mathbf{W}_{xh} + \mathbf{h}_{t-1} \mathbf{W}_{hh} + \mathbf{b}_h)
$$

其结构核心包括：
- $\mathbf{x}_t$：当前时间步的输入
- $\mathbf{h}_{t-1}$：上一时间步的隐藏状态
- $\mathbf{W}_{xh}$：输入到隐藏状态的权重矩阵
- $\mathbf{W}_{hh}$：隐藏状态之间的权重矩阵
- $\mathbf{b}_h$：偏置项
- $\tanh$：非线性激活函数，用于增强模型表达能力

多个时间步串联时，隐藏状态就会随时间演化：


In [ ]:
import torch
import torch.nn.functional as F

def rnn_forward(inputs, h0, W_xh, W_hh, b_h):
    H = h0.shape[1]
    outputs, H_t = [], h0
    for X_t in inputs:  # 遍历时间步
        H_t = torch.tanh(torch.matmul(X_t, W_xh) + torch.matmul(H_t, W_hh) + b_h)
        outputs.append(H_t)
    return torch.stack(outputs), H_t

# 输入：(时间步数, 批大小, 输入维度)
X = torch.randn(5, 3, 10)
h0 = torch.zeros(3, 20)

W_xh = torch.randn(10, 20)
W_hh = torch.randn(20, 20)
b_h = torch.randn(20)

output, hn = rnn_forward(X, h0, W_xh, W_hh, b_h)
print(output.shape)  # (5, 3, 20)
print(hn.shape)      # (3, 20)
